In [3]:
#working on parsing the data using pandas.read_csv()

import pandas as pd
import glob


def parseRtergHtml(html):
    colNames1 = ["Eventname", "oDate", "oTime", "Lat.", "Long.", "Depth", "Me", "Txo",
                 "Ehf", "Ebb", "Mehf", "Ehf/Tr^3", "Nstats", "colon", "SRC", "iMag"]
    try:
        df1 = pd.read_csv(html, names = colNames1, skiprows = 1, nrows = 1, delim_whitespace=True)
        df1["SRC"] = df1.iloc[0]["SRC"][5:7]
        iMagTypeVal = df1.iloc[0]["iMag"].split("=")[0]
        df1.insert(14, "iMagType", iMagTypeVal)
        df1["iMag"] = df1.iloc[0]["iMag"].split("=")[1][:-2]
        oTimeVal = df1["oDate"] + " " + df1["oTime"]
        df1["oTime"] = pd.to_datetime(oTimeVal, utc=True)
        intSkip1 = 0
    except:
        df1 = pd.DataFrame(columns = colNames1)
        intSkip1 = 1
    
    del df1["colon"]
    del df1["oDate"]
    
    colNames2 = ["TACER_HF", "TACER_BB"]
    
    try:
        df2 = pd.read_csv(html, names = colNames2, skiprows = 2, nrows = 1, delim_whitespace=True, usecols=[7,10])
        float(df2["TACER_HF"][0]) # test to see if there is a valid number
        intSkip2 = 0
    except:
        df2 = pd.DataFrame(columns = colNames2)
        intSkip2 = 1

    colNames3 = ["junk", "Comment"]
    colNames4 = ["junk", "mTime"]
    colNames5 = ["junk", "iteration"]
    
    df3 = pd.read_csv(html, names = colNames3, skiprows = 3 - intSkip2 - intSkip1, nrows = 1, engine="python", delimiter="S:")
    del df3["junk"]
    
    f = open(html, "r")
    content = f.read().split("\n")
    count = -1
    for lines in content:
        count += 1
    if content[count] == "":
        count -= 1
    
    df4 = pd.read_csv(html, names = colNames4, skiprows = count - 1, nrows = 1, delimiter="|")
    if df4["junk"][0][-5:-1] == "</a>":
        del df4["junk"]
    else:
        del df4["mTime"]
        df4.rename(columns = {"junk" : "mTime"}, inplace=True)
    df4["mTime"] = pd.to_datetime(df4["mTime"], utc=True)
    
    df5 = pd.read_csv(html, names = colNames5, skiprows = count, nrows = 1, delimiter="=")
    del df5["junk"]
    df5["iteration"][0] = df5.iloc[0]["iteration"].split("<")[0]

    df = pd.concat([df1,df2,df3,df4, df5], axis=1)
    return df

In [5]:
def builddf(htmlfiles):
    df = pd.DataFrame()
    for html in htmlfiles:
        try: 
            df1=parseRtergHtml(html)
            if len(df) == 0:  # first run keeps header
                df=df1
            else:  # otherwise strip it
                df=df.append(df1, ignore_index = True)
            count += 1
        except:
            continue
    return df

#htmlfiles = sorted(glob.glob('rterg_html_outs/[0-9]???????.html'))  # creates a sorted version
#htmlfiles=glob.glob('rterg_html_outs/bad/*.html')

df = builddf(htmlfiles)
df.head()

NameError: name 'htmlfiles' is not defined

In [526]:
print(df.index)

RangeIndex(start=0, stop=4, step=1)
